In [24]:
//input
val fileName1 = "dataset_abstracts/a9000006.txt"
val fileName2 = "dataset_abstracts/a9000031.txt"

val rows1 = sc.textFile(fileName1).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text1 = rows1.reduce(_ + _)

val rows2 = sc.textFile(fileName1).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text2 = rows2.reduce(_ + _)

fileName1 = dataset_abstracts/a9000006.txt
fileName2 = dataset_abstracts/a9000031.txt
rows1 = MapPartitionsRDD[50] at map at <console>:35
text1 = Commercial exploitation over the past two hundred years drovethe great Mysticete whales to near extinction. Variation inthe sizes of populations prior to exploitation, minimalpopulation size during exploitation and current populationsizes permit analyses of the effects of differing levels ofexploitation on species with different biogeographicaldistributions and life-history characteristics. Dr. StephenPalumbi at the University of Hawaii will study the geneticpopulation structure of three whale species in this context,the Humpback Whale, the Gray Whale and the Bowhead Whale. Theeffect of...


Commercial exploitation over the past two hundred years drovethe great Mysticete whales to near extinction. Variation inthe sizes of populations prior to exploitation, minimalpopulation size during exploitation and current populationsizes permit analyses of the effects of differing levels ofexploitation on species with different biogeographicaldistributions and life-history characteristics. Dr. StephenPalumbi at the University of Hawaii will study the geneticpopulation structure of three whale species in this context,the Humpback Whale, the Gray Whale and the Bowhead Whale. Theeffect of...

In [53]:
//k-gram
val k = 5
val k_gram1 = text1.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD1 = sc.parallelize(k_gram1)
val filtered1 = kgramRDD1.distinct()

val k_gram2 = text2.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD2 = sc.parallelize(k_gram2)
val filtered2 = kgramRDD2.distinct()




k = 5
k_gram1 = List((-368554769,""), (-2116654235,""), (1348093184,""), (-713942165,""), (154171951,""), (-1539835203,""), (621014717,""), (-702057327,""), (1061980120,""), (-1210501041,""), (-1777376094,""), (-2146599456,""), (1638764184,""), (2132630876,""), (-558803310,""), (-1704996955,""), (1554848983,""), (-643897612,""), (277879516,""), (1658643371,""), (-239693190,""), (-1096516904,""), (1369629706,""), (2053951917,""), (1404211726,""), (-496476476,""), (66848233,""), (-297738899,""), (1003038978,""), (229228928,""), (-120662840,""), (1068215916,""), (-475916955,""), (-443907787,""), (2075776880,""), (230583223,""), (1330534066,""), (-1120787669,""), (-1584074738,""), (-1702092513,""), (1354781654,""), (-1494390031,""), (-310258093,""), (1911875298,"")...


List((-368554769,""), (-2116654235,""), (1348093184,""), (-713942165,""), (154171951,""), (-1539835203,""), (621014717,""), (-702057327,""), (1061980120,""), (-1210501041,""), (-1777376094,""), (-2146599456,""), (1638764184,""), (2132630876,""), (-558803310,""), (-1704996955,""), (1554848983,""), (-643897612,""), (277879516,""), (1658643371,""), (-239693190,""), (-1096516904,""), (1369629706,""), (2053951917,""), (1404211726,""), (-496476476,""), (66848233,""), (-297738899,""), (1003038978,""), (229228928,""), (-120662840,""), (1068215916,""), (-475916955,""), (-443907787,""), (2075776880,""), (230583223,""), (1330534066,""), (-1120787669,""), (-1584074738,""), (-1702092513,""), (1354781654,""), (-1494390031,""), (-310258093,""), (1911875298,"")...

In [66]:
println(filtered1.count())
println(filtered2.count())
val join = filtered1.leftOuterJoin(filtered2)
val join_n = join.count().toFloat
val union = filtered1.union(filtered2).distinct()
val union_n = union.count().toFloat

val sim = join_n / union_n
val jacc_dist = 1-sim

println(join_n)
println(union_n)

println(sim)
println(jacc_dist)


1049
1049
1049.0
1053.0
0.99620134
0.0037986636


join = MapPartitionsRDD[227] at leftOuterJoin at <console>:37
join_n = 1049.0
union = MapPartitionsRDD[231] at distinct at <console>:39
union_n = 1053.0
sim = 0.99620134
jacc_dist = 0.0037986636


0.0037986636

Now we work with Min-hashing

In [ ]:
//Min-hashing

val data = Array()
data.
